<a href="https://colab.research.google.com/github/data-aleks/UFO_Sightings_python/blob/main/ufo_sightings_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **PT1 Data cleaning of UFO sightings data set** to learn data cleaning using python

**Project goal:**
I want to explore, clean, and analyze UFO sightings from the Kaggle dataset using Python. Learn python data analytics workflow and become comfrotable in using it for data analysis. This is **part 1** of the project where i dive head first in to data cleaning.

**Overview of ufo_dataset.csv:**
*   datetime - date and time information about the sighting
*   city - city where the sighting occured
*   state - state where the sighting occured
*   country - country where the sighting occured
*   shape - shape of the ufo
*   duration (seconds) - duration in seconds of the sighting
*   duration (hours/min) - duration in hours/minutes of teh sighting
*   comments - comment describing sighting
*   date posted - date this posting sighting made public
*   latitude - latitude information
*   longitude - longitude information

**Process steps:**
1.   Getting data
2.   Cleaning data
  *   Dealing with column names, exploring for duplicates
  *   Dealing with missing city and state values based on longitude and latitude using **GeoPy**, **Nominatim**
  *   Dealing with parenthesis in state column
  *   Dealing with mixed formatting of date columns
  *   Dealing with sighting duration columns
  *   Dealing with html entities and non ascii characters in comments
3. Saving cleaned dataset

**Summary of data cleaning:**
*   Original row count: **80332**
*   Cleaned row count: **79561**
*   Records lost: **771**
*   Percentage of records lost: **0.96%**

**Project Summary: Lessons Learned from Data Cleaning**

Through this project, I’ve learned firsthand that data cleaning is far from a simple, one-step process—it requires a blend of different tools, methods, and approaches to handle diverse challenges in the dataset. Each step revealed new complexities, reinforcing the importance of a structured approach to ensure high-quality, usable data.

One of the key techniques I mastered was reverse geocoding missing city and state values using GeoPy and Nominatim, which allowed me to enrich incomplete location data by leveraging latitude and longitude information. This was crucial in reconstructing missing values that would otherwise limit analysis.
Additionally, I tackled the challenge of removing HTML entities from text data—ensuring comments were cleaned of artifacts like &#44 and other encoded characters. Beyond that, handling non-ASCII special characters required Unicode normalization techniques to make the text readable and consistent.

Another fundamental learning was the complexity of time-based fields. Cleaning duration data meant understanding how to convert and format numerical values correctly, balancing precision with usability. I also dealt with mixed formatting issues in datetime fields to standardize the structure for reliable analysis.

Throughout the project, I approached data validation carefully by comparing the original dataset with the cleaned version, tracking lost records, and calculating the percentage reduction (0.96% lost records). This helped ensure that my cleaning process did not unintentionally remove valuable information.
Ultimately, this project reinforced the fact that data cleaning isn’t just about removing errors—it’s about making data functional, reliable, and insightful. By working through real-world data inconsistencies, I’ve gained confidence in applying Python-based data cleaning workflows, which will be foundational in my continued journey into data analytics.








# **Step 1. Getting data**

Let's import our python toolset to be able to work with our data.

In [ ]:
import pandas as pd
import numpy as np

Let's read data from our CSV file.

In [ ]:
# Read the csv file using pandas read_csv
df = pd.read_csv('./drive/MyDrive/datasets/ufo_dataset.csv')

<ipython-input-99-4217738253>:2: DtypeWarning: Columns (5,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./drive/MyDrive/datasets/ufo_dataset.csv')


Let's see first 10 lines of our dataset.

In [ ]:
# Get first 10 records of the dataset
df.head(10)

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.941111
1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.38421,-98.581082
2,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.2,-2.916667
3,10/10/1956 21:00,edna,tx,us,circle,20,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.9783333,-96.645833
4,10/10/1960 20:00,kaneohe,hi,us,light,900,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.4180556,-157.803611
5,10/10/1961 19:00,bristol,tn,us,sphere,300,5 minutes,My father is now 89 my brother 52 the girl wit...,4/27/2007,36.595,-82.188889
6,10/10/1965 21:00,penarth (uk/wales),NaN,gb,circle,180,about 3 mins,penarth uk circle 3mins stayed 30ft above m...,2/14/2006,51.434722,-3.180000
7,10/10/1965 23:45,norwalk,ct,us,disk,1200,20 minutes,A bright orange color changing to reddish colo...,10/02/1999,41.1175,-73.408333
8,10/10/1966 20:00,pell city,al,us,disk,180,3 minutes,Strobe Lighted disk shape object observed clos...,3/19/2009,33.5861111,-86.286111
9,10/10/1966 21:00,live oak,fl,us,disk,120,several minutes,Saucer zaps energy from powerline as my pregna...,05/11/2005,30.2947222,-82.984167


Let's see general information about our dataset.

In [ ]:
# General information about the dataset using df.info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80332 entries, 0 to 80331
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   datetime              80332 non-null  object 
 1   city                  80332 non-null  object 
 2   state                 74535 non-null  object 
 3   country               70662 non-null  object 
 4   shape                 78400 non-null  object 
 5   duration (seconds)    80332 non-null  object 
 6   duration (hours/min)  80332 non-null  object 
 7   comments              80317 non-null  object 
 8   date posted           80332 non-null  object 
 9   latitude              80332 non-null  object 
 10  longitude             80332 non-null  float64
dtypes: float64(1), object(10)
memory usage: 6.7+ MB


**What can we see?**
In total there is 80332 entries in this dataset. This dataset contains records of UFO sightings around the world. Some of the columns have missing values like **country, state, shape, comments and longitude**

# **Step 2. Cleaning Data**

In this step we will look at our data whether we are renaming columns, removing duplicates, changing types or dealing with missing values our mission during this step is to ensure the data is in it's best shape for us to work with.

## **Step 2.1 Dealing with column names, exploring for duplicates**

First we will create a copy of our data frame and store it in a variable df_processing.

In [ ]:
# Create a copy of the imported dataframe
df_processing = df.copy()

Let's look at our column names.

In [ ]:
# Retrieve column names from the dataset
df_processing.columns

Index(['datetime', 'city', 'state', 'country', 'shape', 'duration (seconds)',
       'duration (hours/min)', 'comments', 'date posted', 'latitude',
       'longitude '],
      dtype='object')

We would wan't to **remove spaces** in our column names and **symbols** as this will make our life easier when working with column names. We will also remove a trailing space in our longitude column

In [ ]:
# Lets use rename and dictionary to rename multiple columns at the same time
df_processing.rename(columns={
    'duration (seconds)': 'duration_seconds',
    'duration (hours/min)': 'duration_hours_min',
    'date posted': 'date_posted',
    'longitude ': 'longitude'
}, inplace=True)

# Let's check the resulting column names
df_processing.columns

Index(['datetime', 'city', 'state', 'country', 'shape', 'duration_seconds',
       'duration_hours_min', 'comments', 'date_posted', 'latitude',
       'longitude'],
      dtype='object')

Lets also check our dataset for any duplicates.

In [ ]:
# Display number of duplicated records
df_processing.duplicated().sum()

np.int64(0)

Lets check our data frame for missing values.

In [ ]:
# Display sum of missing values
df_processing.isnull().sum()

,0
datetime,0
city,0
state,5797
country,9670
shape,1932
duration_seconds,0
duration_hours_min,0
comments,15
date_posted,0
latitude,0


We can see that our **state column** is **missing 5797 values**, **country** is **missing 9670 values**, **shape** is **missing 1932 value**s and **comments** and **longitude** both **missing 1 value**.

## **Step 2.2 Dealing with missing city, state values using GeoPy, Nominatim**

Lets start by looking at our records that are missing country and see if we are able to fix our data by filling the missing values.

In [ ]:
# Select records that are missing country
df_processing[df_processing['country'].isnull()]

,datetime,city,state,country,shape,duration_seconds,duration_hours_min,comments,date_posted,latitude,longitude
1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.38421,-98.581082
18,10/10/1973 23:00,bermuda nas,NaN,NaN,light,20,20 sec.,saw fast moving blip on the radar scope thin w...,01/11/2002,32.364167,-64.678611
29,10/10/1979 22:00,saddle lake (canada),ab,NaN,triangle,270,4.5 or more min.,Lights far above&#44 that glance; then flee f...,1/19/2005,53.970571,-111.689885
35,10/10/1982 07:00,gisborne (new zealand),NaN,NaN,disk,120,2min,gisborne nz 1982 wainui beach to sponge bay,01/11/2002,-38.662334,178.017649
40,10/10/1986 20:00,holmes/pawling,ny,NaN,chevron,180,3 minutes,Football Field Sized Chevron with bright white...,10/08/2007,41.523427,-73.646795
...,...,...,...,...,...,...,...,...,...,...,...
80238,09/09/2009 14:15,broomfield?lafayette,co,NaN,rectangle,120.0,2 min,Large&#44 rectangular object seen flying in br...,12/12/2009,39.993596,-105.089706
80244,09/09/2009 20:17,lyman,me,NaN,light,600.0,10 mins,Two lights ran across the sky&#44 as bright as...,12/12/2009,43.505096,-70.637968
80319,09/09/2013 20:15,clifton,nj,NaN,other,3600.0,~1hr+,Luminous line seen in New Jersey sky.,9/30/2013,40.858433,-74.163755
80322,09/09/2013 21:00,aleksandrow (poland),NaN,NaN,light,15.0,15 seconds,Two points of light following one another in a...,9/30/2013,50.465843,22.891814


In [ ]:
# Select records that are missing country
df_processing[df_processing['state'].isnull()]

,datetime,city,state,country,shape,duration_seconds,duration_hours_min,comments,date_posted,latitude,longitude
2,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.2,-2.916667
6,10/10/1965 21:00,penarth (uk/wales),NaN,gb,circle,180,about 3 mins,penarth uk circle 3mins stayed 30ft above m...,2/14/2006,51.434722,-3.180000
18,10/10/1973 23:00,bermuda nas,NaN,NaN,light,20,20 sec.,saw fast moving blip on the radar scope thin w...,01/11/2002,32.364167,-64.678611
20,10/10/1974 21:30,cardiff (uk/wales),NaN,gb,disk,1200,20 minutes,back in 1974 I was 19 at the time and lived i...,02/01/2007,51.5,-3.200000
24,10/10/1976 22:00,stoke mandeville (uk/england),NaN,gb,cigar,3,3 seconds,White object over Buckinghamshire UK.,12/12/2009,51.783333,-0.783333
...,...,...,...,...,...,...,...,...,...,...,...
80217,09/09/2007 19:01,melbourne (australia),NaN,au,circle,600.0,10 min,Hostile,10/08/2007,-37.813938,144.963425
80234,09/09/2009 03:14,aberdeen (uk/scotland),NaN,gb,light,6.0,6 seconds,Bright light seen over Aberdeen&#44 Scotland&#...,12/12/2009,57.166667,-2.666667
80254,09/09/2009 21:15,nottinghamshire (uk/england),NaN,gb,fireball,600.0,10 mins,resembled orange flame imagine a transparent h...,12/12/2009,53.166667,-1.000000
80255,09/09/2009 21:38,kaiserlautern (germany),NaN,de,light,40.0,about 40 seconds,2 white lights over Kaiserslautern&#44 ramstei...,12/12/2009,49.45,7.750000


As we can see there is alot of missing data. Some records contain the country in the city column some don't. We could possibly extract this data an fill in the missing values. However we do have longitude and latitude data an perhaps should be able to use some kind of geolocation API to replace the missing data.

After some digging we could potentially use the longitude and latitude data to fix our missing country, state values a by using something like **geopy** with it's **Nominatim** module.

**!! WARNING !! - DO NOT RUN BELOW CODE** Below code will execute a function that will reverse geocode city, country, state. We have to adhere with api guidlines an ensure we make calls to API with 1 second intervals to avoid being denied access. There is **12211 records** that are missing values, this will take **12211 seconds** to complete which is about **4 hours**. I've exectued this code and have save the data to a separate dataset called **df_geo_processed** and will be using it in the future steps

In [ ]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
import time
from tqdm import tqdm

# This python script is to allow me to test the reverse geolocation

# --- Initialize Nominatim geolocator ---
# IMPORTANT: Replace "ufo_dataset_project_your_name_or_email" with a unique identifier
geolocator = Nominatim(user_agent="ufo_dataset_project_your_name_or_email")

# --- Geocoding Function (MODIFIED) ---
def get_location_details_smart(row):
    """
    Attempts to reverse geocode missing city, state, or country values using latitude/longitude.
    Only proceeds if latitude and longitude are available and at least one of city, state, or country is missing.
    Prioritizes country_code for country and searches for common administrative divisions for state.
    """
    current_city = row['city'] if pd.notna(row['city']) else None
    current_country = row['country'] if pd.notna(row['country']) else None
    current_state = row['state'] if pd.notna(row['state']) else None
    current_latitude = row['latitude'] if pd.notna(row['latitude']) else None
    current_longitude = row['longitude'] if pd.notna(row['longitude']) else None

    result_city = current_city
    result_state = current_state
    result_country = current_country # This will hold the abbreviation if found

    if (current_latitude is not None and current_longitude is not None and
        (current_city is None or current_state is None or current_country is None)):

        try:
            # IMPORTANT: Keep this delay! 1 second per request is crucial for Nominatim's fair usage policy.
            time.sleep(1)
            location = geolocator.reverse(f"{current_latitude}, {current_longitude}", language='en')

            if location and location.raw and 'address' in location.raw:
                address = location.raw['address']

                # --- Handle City ---
                if result_city is None:
                    # Prioritize common city-like keys
                    result_city = address.get('city') or \
                                  address.get('town') or \
                                  address.get('village') or \
                                  address.get('hamlet')

                # --- Handle State (or equivalent) ---
                if result_state is None:
                    # Prioritize common administrative divisions for 'state'
                    result_state = address.get('state') or \
                                   address.get('province') or \
                                   address.get('region') or \
                                   address.get('county') # county might be too granular for 'state', but can be a fallback

                # --- Handle Country (with Abbreviation) ---
                if result_country is None:
                    # Prefer country_code (e.g., 'US', 'GB') if available, otherwise use full 'country' name.
                    # Convert to lowercase for consistency as per original dataset abbreviations (AU, GB).
                    country_info = address.get('country_code', address.get('country'))
                    if country_info:
                        result_country = country_info.upper() # Changed to .upper() as per previous instructions

        except (GeocoderTimedOut, GeocoderServiceError) as e:
            # print(f"Error reverse geocoding {current_latitude}, {current_longitude}: {e}")
            pass
        except Exception as e:
            # print(f"An unexpected error occurred during reverse geocoding {current_latitude}, {current_longitude}: {e}")
            pass

    return {'city': result_city, 'state': result_state, 'country': result_country}


# Identify records for geocoding
print("Identifying records for geocoding...")
rows_to_process_mask = (df_processing['latitude'].notna()) & \
                       (df_processing['longitude'].notna()) & \
                       ((df_processing['city'].isna()) | \
                        (df_processing['state'].isna()) | \
                        (df_processing['country'].isna()))

df_needs_geocoding = df_processing[rows_to_process_mask].copy()
print(f"Identified {len(df_needs_geocoding)} records needing geocoding.")

# --- Test Sample Code ---
# df_test_sample = df_needs_geocoding.sample(n=500, random_state=42)
# test_sample_indices = df_test_sample.index.tolist()
# total_records_to_process = len(df_test_sample)

# --- Full Run: Process all identified records ---
total_records_to_process = len(df_needs_geocoding) # Now targeting the full set
successfully_geocoded_count = 0

print(f"\n**Starting reverse geocoding for {total_records_to_process} records.** This may take a while (approx. {total_records_to_process} seconds).")

if total_records_to_process > 0:
    pbar = tqdm(df_needs_geocoding.iterrows(), total=total_records_to_process,
                desc=f"Geocoding (Filled: 0/{total_records_to_process}, Remaining: {total_records_to_process})")

    for index, row in pbar:
        original_city = df_processing.loc[index, 'city']
        original_state = df_processing.loc[index, 'state']
        original_country = df_processing.loc[index, 'country']

        geocoded_values = get_location_details_smart(row)

        # Update df_processing directly based on the original index
        df_processing.loc[index, 'city'] = geocoded_values['city']
        df_processing.loc[index, 'state'] = geocoded_values['state']
        df_processing.loc[index, 'country'] = geocoded_values['country']

        if (pd.isna(original_city) and pd.notna(df_processing.loc[index, 'city'])) or \
           (pd.isna(original_state) and pd.notna(df_processing.loc[index, 'state'])) or \
           (pd.isna(original_country) and pd.notna(df_processing.loc[index, 'country'])):
            successfully_geocoded_count += 1

        pbar.set_description(f"Geocoding (Filled: {successfully_geocoded_count}/{total_records_to_process}, Remaining: {total_records_to_process - successfully_geocoded_count})")
else:
    print("No records found that meet the criteria for reverse geocoding (i.e., having lat/lon but missing city/state/country). No geocoding performed.")

print("\n" + "="*50 + "\n")
print("--- Geocoding Complete ---")
print(df_processing.head()) # Show a glimpse of the updated DataFrame
print(f"\n**Summary:** Successfully filled at least one missing field (city/state/country) for **{successfully_geocoded_count}** records out of **{total_records_to_process}** that met the geocoding criteria.")

# ---Test Sample Saving Code ---
# df_updated_test_records = df_processing.loc[test_sample_indices].copy()
# df_test_sample_original = df.loc[test_sample_indices].copy()
# df_comparison = pd.concat([
#     df_test_sample_original.add_suffix('_original'),
#     df_updated_test_records.add_suffix('_updated')
# ], axis=1)
# df_updated_test_records.to_csv('./dataset/geocoded_test_sample_updated.csv', index=True)
# print(f"\nSaved updated test records to: ./dataset/geocoded_test_sample_updated.csv")
# df_comparison.to_csv('./dataset/geocoded_test_sample_comparison.csv', index=True)
# print(f"Saved comparison of original vs. updated test records to: ./dataset/geocoded_test_sample_comparison.csv")

# --- Final Save of the fully processed DataFrame ---
output_file_path = './drive/MyDrive/dataset/df_geo_processed.csv'
print(f"\nSaving the full geocoded DataFrame to {output_file_path}...")
df_processing.to_csv(output_file_path, index=False) # index=False is important
print("Save complete!")

Let's take a look at the result of us using reverse geo on our data set to fix missing values.

In [ ]:
# Load up data set
df_geo_processed = pd.read_csv('./drive/MyDrive/datasets/df_geo_processed.csv')

<ipython-input-44-3767833595>:2: DtypeWarning: Columns (5,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_geo_processed = pd.read_csv('./drive/MyDrive/datasets/df_geo_processed.csv')


First let's see our original

In [ ]:
# First lets see our original
df_processing.isnull().sum()

NameError: name 'df_processing' is not defined

And now lets see our dataset after making a call to geopy to reverse geo locate

In [ ]:
# Dataset after using geopy
df_geo_processed.isnull().sum()
df_geo_processed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80332 entries, 0 to 80331
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   datetime            80332 non-null  object 
 1   city                80332 non-null  object 
 2   state               80001 non-null  object 
 3   country             80231 non-null  object 
 4   shape               78400 non-null  object 
 5   duration_seconds    80332 non-null  object 
 6   duration_hours_min  80332 non-null  object 
 7   comments            80317 non-null  object 
 8   date_posted         80332 non-null  object 
 9   latitude            80332 non-null  object 
 10  longitude           80332 non-null  float64
dtypes: float64(1), object(10)
memory usage: 6.7+ MB


As we can see it we have reduced number of records missing state/country quite considerably, however this did not completely remove the issue completely and we still have some records that are missing values.

In [ ]:
df_geo_processed[df_geo_processed['state'].isnull()]

,datetime,city,state,country,shape,duration_seconds,duration_hours_min,comments,date_posted,latitude,longitude
296,10/11/1997 22:00,hafnarfjordur (iceland),NaN,IS,sphere,300,5 min,playing with a jet,06/12/2008,64.066667,-21.950000
515,10/01/1970 23:00,indian ocean (usn vessel),NaN,NaN,light,240,3-4 minutes,Bright object seemingly appeared out of nowher...,07/08/2004,-33.137551,81.826172
889,10/01/2008 21:00,yerevan (armenia),NaN,AM,unknown,60,1 minute,Red light appears in night sky and disappears.,5/13/2012,40.183333,44.516667
1051,10/12/2002 19:13,australia (rural),NaN,AU,other,420,7 minutes,This was a very close encounter with unidenti...,08/12/2008,-25.274398,133.775136
1332,10/13/2006 00:02,berlin (germany),NaN,de,fireball,120,1-2 minutes,7 shooting lights&#44 followed by a formation&...,10/30/2006,52.516667,13.400000
...,...,...,...,...,...,...,...,...,...,...,...
78598,9/30/1995 23:15,canberra (australia),NaN,au,light,60.0,1 minute,Strange light seen over Canberra Australia,10/27/2004,-35.27603,149.134350
78686,9/30/2005 19:35,ulaanbaatar (outer mongolia),NaN,MN,disk,180.0,3 minuts,UFO over Ulaanbaatar&#44 Mongolia,10/11/2005,47.92,106.920000
78896,09/03/2004 03:30,akureyri (iceland),NaN,IS,light,3600.0,1 hour,Car buzzed by a bright object,09/09/2004,65.683333,-18.100000
78898,09/03/2004 14:54,busan (south korea),NaN,KR,chevron,2.0,seconds,It has dark brown color&#44 an empennage-shape...,09/09/2004,35.179554,129.075642


Lets explore records that are missing both country and state

In [ ]:
df_geo_processed[df_geo_processed['country'].isnull() & df_geo_processed['state'].isnull()]

,datetime,city,state,country,shape,duration_seconds,duration_hours_min,comments,date_posted,latitude,longitude
515,10/01/1970 23:00,indian ocean (usn vessel),NaN,NaN,light,240,3-4 minutes,Bright object seemingly appeared out of nowher...,07/08/2004,-33.137551,81.826172
1740,10/15/1968 21:30,pacific ocean (1500mi.sw of u.s.mainland),NaN,NaN,circle,30,30 sec.,Bright&#44 white soundless orb with no trajeco...,09/12/2003,-8.783195,-124.508523
3282,10/20/2008 02:00,indian ocean,NaN,NaN,unknown,300,5 minuts,at night in the middle of the ocean ( a light ...,8/27/2009,-33.137551,81.826172
4212,10/24/1995 02:00,tyrrhenian sea,NaN,NaN,sphere,30,30sec,blue colour sphere was obsereved from containe...,7/16/2006,40.076986,11.343106
5363,10/29/2010 21:00,indian ocean,NaN,NaN,fireball,5400,1.5 hrs,During the routine bridge watch at sea&#44 on ...,11/21/2010,-33.137551,81.826172
...,...,...,...,...,...,...,...,...,...,...,...
74542,9/15/1966 01:30,pacific ocean (western),NaN,NaN,unknown,300.0,5 mis.,object in water 2 feet from boat made a straig...,10/08/2007,-8.783195,-124.508523
75541,9/18/2005 14:00,atlantic ocean,NaN,NaN,disk,60.0,1 minute,lenticular cloud to disc,10/11/2005,-14.599413,-28.673147
76026,9/20/1988 13:00,atlantic ocean,NaN,NaN,unknown,20.0,20 seconds,The craft was visible at different positions f...,05/11/2005,-14.599413,-28.673147
76282,9/21/1988 03:00,atlantic ocean (middle),NaN,NaN,fireball,15.0,15 seconds,The light clearly lit up the bow of the vessel...,05/11/2005,-14.599413,-28.673147


These records are records of sighting over the oceans/seas and represent a small percentage of our dataset and will not impact our analysis. There is couple of options we can do we can ether remove the records, create a separate dataset for maritime sightings etc. As this is only a small percentage of records we will remove them from our dataset.

In [ ]:
# Lets copy our dataset and remove records where both 'state' and 'country' are missing
df_maritime_scrub = df_geo_processed.copy()

df_maritime_scrub = df_maritime_scrub[~(df_maritime_scrub['state'].isnull() & df_maritime_scrub['country'].isnull())]

df_maritime_scrub.reset_index(drop=True, inplace=True)

# Let's see the impact
df_maritime_scrub.isnull().sum()
df_maritime_scrub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80264 entries, 0 to 80263
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   datetime            80264 non-null  object 
 1   city                80264 non-null  object 
 2   state               80001 non-null  object 
 3   country             80231 non-null  object 
 4   shape               78335 non-null  object 
 5   duration_seconds    80264 non-null  object 
 6   duration_hours_min  80264 non-null  object 
 7   comments            80250 non-null  object 
 8   date_posted         80264 non-null  object 
 9   latitude            80264 non-null  object 
 10  longitude           80264 non-null  float64
dtypes: float64(1), object(10)
memory usage: 6.7+ MB


In [ ]:
df_maritime_scrub[df_maritime_scrub['state'].isnull()]

,datetime,city,state,country,shape,duration_seconds,duration_hours_min,comments,date_posted,latitude,longitude
296,10/11/1997 22:00,hafnarfjordur (iceland),NaN,IS,sphere,300,5 min,playing with a jet,06/12/2008,64.066667,-21.950000
888,10/01/2008 21:00,yerevan (armenia),NaN,AM,unknown,60,1 minute,Red light appears in night sky and disappears.,5/13/2012,40.183333,44.516667
1050,10/12/2002 19:13,australia (rural),NaN,AU,other,420,7 minutes,This was a very close encounter with unidenti...,08/12/2008,-25.274398,133.775136
1331,10/13/2006 00:02,berlin (germany),NaN,de,fireball,120,1-2 minutes,7 shooting lights&#44 followed by a formation&...,10/30/2006,52.516667,13.400000
1879,10/15/1990 21:30,taipei city (taiwan),NaN,TW,circle,30,30 second,7 luminous objects&#44 circle shapes; 6 object...,10/12/2001,25.091075,121.559834
...,...,...,...,...,...,...,...,...,...,...,...
78530,9/30/1995 23:15,canberra (australia),NaN,au,light,60.0,1 minute,Strange light seen over Canberra Australia,10/27/2004,-35.27603,149.134350
78618,9/30/2005 19:35,ulaanbaatar (outer mongolia),NaN,MN,disk,180.0,3 minuts,UFO over Ulaanbaatar&#44 Mongolia,10/11/2005,47.92,106.920000
78828,09/03/2004 03:30,akureyri (iceland),NaN,IS,light,3600.0,1 hour,Car buzzed by a bright object,09/09/2004,65.683333,-18.100000
78830,09/03/2004 14:54,busan (south korea),NaN,KR,chevron,2.0,seconds,It has dark brown color&#44 an empennage-shape...,09/09/2004,35.179554,129.075642


**!! WARNING !!** - ** DO NOT RUN THIS ** - This script will go through remainder of missing records and will has about **5 minutes to execute**. As we can see we still have some missing records, this might be due to these countries not having State etc. Let's modify our script to possibly address these and return a wider result when it comes to these state values like administrative area or political union.

In [ ]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
import time
from tqdm import tqdm

# This python script is to allow me to test the reverse geolocation on a local machine

# --- Initialize Nominatim geolocator ---
# IMPORTANT: Replace "ufo_dataset_project_your_name_or_email" with a unique identifier
geolocator = Nominatim(user_agent="ufo_dataset_project_your_name_or_email")

# --- Geocoding Function (MODIFIED) ---
def get_location_details_smart(row):
    """
    Attempts to reverse geocode missing city, state, or country values using latitude/longitude.
    Only proceeds if latitude and longitude are available and at least one of city, state, or country is missing.
    Prioritizes country_code for country and searches for common administrative divisions for state.
    Includes additional administrative keys and a final fallback for 'state'.
    """
    current_city = row['city'] if pd.notna(row['city']) else None
    current_country = row['country'] if pd.notna(row['country']) else None
    current_state = row['state'] if pd.notna(row['state']) else None
    current_latitude = row['latitude'] if pd.notna(row['latitude']) else None
    current_longitude = row['longitude'] if pd.notna(row['longitude']) else None

    result_city = current_city
    result_state = current_state
    result_country = current_country # This will hold the abbreviation if found

    if (current_latitude is not None and current_longitude is not None and
        (current_city is None or current_state is None or current_country is None)):

        try:
            # IMPORTANT: Keep this delay! 1 second per request is crucial for Nominatim's fair usage policy.
            time.sleep(1)
            location = geolocator.reverse(f"{current_latitude}, {current_longitude}", language='en')

            if location and location.raw and 'address' in location.raw:
                address = location.raw['address']

                # --- Handle City ---
                if result_city is None:
                    # Prioritize common city-like keys
                    result_city = address.get('city') or \
                                  address.get('town') or \
                                  address.get('village') or \
                                  address.get('hamlet') or \
                                  address.get('suburb') # Added suburb as a city-like fallback

                # --- Handle State (or equivalent) ---
                if result_state is None:
                    # Prioritize common administrative divisions for 'state', including more granular/generic ones
                    result_state = address.get('state') or \
                                   address.get('province') or \
                                   address.get('region') or \
                                   address.get('state_district') or \
                                   address.get('county') or \
                                   address.get('administrative_area') or \
                                   address.get('political_union') # Very generic, might be useful for some regions

                # --- Handle Country (with Abbreviation) ---
                if result_country is None:
                    # Prefer country_code (e.g., 'US', 'GB') if available, otherwise use full 'country' name.
                    # Convert to uppercase for consistency as per original dataset abbreviations (AU, GB).
                    country_info = address.get('country_code', address.get('country'))
                    if country_info:
                        result_country = country_info.upper()

        except (GeocoderTimedOut, GeocoderServiceError) as e:
            # print(f"Error reverse geocoding {current_latitude}, {current_longitude}: {e}")
            pass
        except Exception as e:
            # print(f"An unexpected error occurred during reverse geocoding {current_latitude}, {current_longitude}: {e}")
            pass

    # --- FINAL FALLBACK FOR STATE ---
    # If state is still missing but city was found, use the city name as the state.
    # This is a judgment call for places where the city is the primary administrative unit.
    if result_state is None and result_city is not None:
        result_state = result_city

    return {'city': result_city, 'state': result_state, 'country': result_country}

# Identify records for geocoding
print("Identifying records for geocoding...")
# IMPORTANT: Ensure 'df_processing' is used here, not 'df_maritime_scrub' unless that's your specific setup
rows_to_process_mask = (df_maritime_scrub['latitude'].notna()) & \
                       (df_maritime_scrub['longitude'].notna()) & \
                       ((df_maritime_scrub['city'].isna()) | \
                        (df_maritime_scrub['state'].isna()) | \
                        (df_maritime_scrub['country'].isna()))

df_needs_geocoding_rerun = df_maritime_scrub[rows_to_process_mask].copy()
print(f"Identified {len(df_needs_geocoding_rerun)} records needing geocoding.")

total_records_to_process = len(df_needs_geocoding_rerun) # Now targeting the full set
successfully_geocoded_count = 0

print(f"\n**Starting reverse geocoding for {total_records_to_process} records.** This may take a while (approx. {total_records_to_process} seconds).")

if total_records_to_process > 0:
    pbar = tqdm(df_needs_geocoding_rerun.iterrows(), total=total_records_to_process,
                desc=f"Geocoding (Filled: 0/{total_records_to_process}, Remaining: {total_records_to_process})")

    for index, row in pbar:
        original_city = df_maritime_scrub.loc[index, 'city'] # Use df_processing here for comparison
        original_state = df_maritime_scrub.loc[index, 'state'] # Use df_processing here for comparison
        original_country = df_maritime_scrub.loc[index, 'country'] # Use df_processing here for comparison

        geocoded_values = get_location_details_smart(row)

        # Update df_processing directly based on the original index
        df_maritime_scrub.loc[index, 'city'] = geocoded_values['city']
        df_maritime_scrub.loc[index, 'state'] = geocoded_values['state']
        df_maritime_scrub.loc[index, 'country'] = geocoded_values['country']

        if (pd.isna(original_city) and pd.notna(df_maritime_scrub.loc[index, 'city'])) or \
           (pd.isna(original_state) and pd.notna(df_maritime_scrub.loc[index, 'state'])) or \
           (pd.isna(original_country) and pd.notna(df_maritime_scrub.loc[index, 'country'])):
            successfully_geocoded_count += 1

        pbar.set_description(f"Geocoding (Filled: {successfully_geocoded_count}/{total_records_to_process}, Remaining: {total_records_to_process - successfully_geocoded_count})")
else:
    print("No records found that meet the criteria for reverse geocoding (i.e., having lat/lon but missing city/state/country). No geocoding performed.")

print("\n" + "="*50 + "\n")
print("--- Geocoding Complete ---")
print(df_maritime_scrub.head()) # Show a glimpse of the updated DataFrame
print(f"\n**Summary:** Successfully filled at least one missing field (city/state/country) for **{successfully_geocoded_count}** records out of **{total_records_to_process}** that met the geocoding criteria.")


# # --- Final Save of the fully processed DataFrame ---
# output_file_path = './drive/MyDrive/datasets/df_geo_processed_rerun.csv'
# print(f"\nSaving the full geocoded DataFrame to {output_file_path}...")
# df_processing.to_csv(output_file_path, index=False) # index=False is important
# print("Save complete!")

Loading dataset...
Identifying records for geocoding...
Identified 296 records needing geocoding.

**Starting reverse geocoding for 296 records.** This may take a while (approx. 296 seconds).


Geocoding (Filled: 159/296, Remaining: 137):  54%|█████▍    | 160/296 [03:18<03:08,  1.39s/it]/usr/local/lib/python3.11/dist-packages/geopy/point.py:455: UserWarning: Latitude normalization has been prohibited in the newer versions of geopy, because the normalized value happened to be on a different pole, which is probably not what was meant. If you pass coordinates as positional args, please make sure that the order is (latitude, longitude) or (y, x) in Cartesian terms.
  return cls(latitude, longitude, altitude)
Geocoding (Filled: 288/296, Remaining: 8): 100%|██████████| 296/296 [06:07<00:00,  1.24s/it]



--- Geocoding Complete ---
           datetime                  city    state country     shape  \
0  10/10/1949 20:30            san marcos       tx      us  cylinder   
1  10/10/1949 21:00          lackland afb       tx      US     light   
2  10/10/1955 17:00  chester (uk/england)  England      gb    circle   
3  10/10/1956 21:00                  edna       tx      us    circle   
4  10/10/1960 20:00               kaneohe       hi      us     light   

  duration_seconds duration_hours_min  \
0             2700         45 minutes   
1             7200            1-2 hrs   
2               20         20 seconds   
3               20           1/2 hour   
4              900         15 minutes   

                                            comments date_posted    latitude  \
0  This event took place in early fall around 194...   4/27/2004  29.8830556   
1  1949 Lackland AFB&#44 TX.  Lights racing acros...  12/16/2005    29.38421   
2  Green/Orange circular disc over Chester&#44 En..

**WARNING** -- Do not run this code as this is simply to save the dataset that rerun of our reverse geolocation code.

In [ ]:
# Lets save the dataset to to avoid having to rerun the operation
# Save the resulting dataset
# output_file_path = './drive/MyDrive/datasets/df_geo_maritime_scrub_rerun.csv'
# df_maritime_scrub.to_csv(output_file_path, index=False) # index=False is important


Well use the CSV file with the rerun of our reverse geo code and call it **df_maritime_scrub_rerun**

In [ ]:
# We use the previously saved dataset
df_maritime_scrub_rerun = pd.read_csv('./drive/MyDrive/datasets/df_geo_maritime_scrub_rerun.csv')

<ipython-input-2-2766294824>:2: DtypeWarning: Columns (5,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_maritime_scrub_rerun = pd.read_csv('./drive/MyDrive/datasets/df_geo_maritime_scrub_rerun.csv')


Lets see the impact of our previous code. We have 8 records remaining that have missing values in country column. Let's investigate further.

In [ ]:
df_maritime_scrub_rerun.isnull().sum()

df_maritime_scrub_rerun.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80264 entries, 0 to 80263
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   datetime            80264 non-null  object 
 1   city                80264 non-null  object 
 2   state               80264 non-null  object 
 3   country             80252 non-null  object 
 4   shape               78335 non-null  object 
 5   duration_seconds    80264 non-null  object 
 6   duration_hours_min  80264 non-null  object 
 7   comments            80250 non-null  object 
 8   date_posted         80264 non-null  object 
 9   latitude            80264 non-null  object 
 10  longitude           80264 non-null  float64
dtypes: float64(1), object(10)
memory usage: 6.7+ MB


Let's view the 8 records missing the values. We can see these records have state however are recorded in sea/ocean which we previously removed from our dataset. As we will not require this records and the records represent a small portion of our dataset i think we can safely remove them.

In [ ]:
df_maritime_scrub_rerun[df_maritime_scrub_rerun['country'].isnull()]

,datetime,city,state,country,shape,duration_seconds,duration_hours_min,comments,date_posted,latitude,longitude
4270,10/24/2004 23:27,na,oh,NaN,cylinder,5,2-5 seconds,Bright flash lightens up night sky&#44 object ...,10/27/2004,-21.846665,19.188005
26947,2/16/2009 20:30,okahandja (namibia),Otjozondjupa,NaN,light,240,4 min,glowing orange round circle next to the moon&#...,11/21/2010,-21.983333,16.916667
30340,03/10/2013 20:00,pacific ocean (near huntington beach),ca,NaN,formation,360,6 minutes,Amber orbs journalist and witness video and lo...,5/15/2013,-8.783195,-124.508523
43748,5/22/1974 05:30,mescalero indian reservation,nm,NaN,rectangle,180,two hours,Huge rectangular object emmitting intense whit...,4/18/2012,33q.200088,-105.624152
43842,5/22/2009 12:50,windhoek (namibia),Khomas,NaN,fireball,1200,20 min,This was a flame coloured moving object in the...,06/09/2009,-22.57,17.083611
49228,6/15/2004 14:24,pacific ocean (in-flight sighting),ca,NaN,oval,30,30 seconds,An oval shaped ship with a metallic color shot...,4/16/2005,-8.783195,-124.508523
51779,6/25/2012 22:00,atlantic ocean,fl,NaN,circle,2700,45 minutes,Glowing Orange UFO at Sea.,07/03/2013,-14.599413,-28.673147
55759,07/12/2008 23:00,swakopmund (namibia),Erongo Region,NaN,light,120,2 minutes,Last night (12 July 2008 23:00hrs) I witnessed...,08/12/2008,-22.683333,14.533333
60510,7/29/2003 24:00,atlantic ocean (canada),ns,NaN,circle,900,15 min,baseball size lights&#44one behind the other&#...,7/16/2006,-14.599413,-28.673147
63129,07/06/2007 23:00,atlantic ocean,fl,NaN,circle,600,10 minutes,July 6&#44 2007 Aboard the Carnival Liberty&#...,08/07/2007,-14.599413,-28.673147


Let's remove these records

In [ ]:
# Remove missing values records
df_maritime_scrub_rerun.dropna(subset=['country'], inplace=True)

# Check our dataset
df_maritime_scrub_rerun.isnull().sum()
df_maritime_scrub_rerun.info()

<class 'pandas.core.frame.DataFrame'>
Index: 80252 entries, 0 to 80263
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   datetime            80252 non-null  object 
 1   city                80252 non-null  object 
 2   state               80252 non-null  object 
 3   country             80252 non-null  object 
 4   shape               78323 non-null  object 
 5   duration_seconds    80252 non-null  object 
 6   duration_hours_min  80252 non-null  object 
 7   comments            80238 non-null  object 
 8   date_posted         80252 non-null  object 
 9   latitude            80252 non-null  object 
 10  longitude           80252 non-null  float64
dtypes: float64(1), object(10)
memory usage: 7.3+ MB


## **Step 2.3 Dealing with shape values and comments. Removing maritime records**

Dealing with missing values in shape and comments should be relatively straightforward. If the shape is missing we will replace missing values with Unknown. If the comments are missing we will replace values with No comment

In [ ]:
# Replace missing shape values with Unkown
df_maritime_scrub_rerun['shape'].fillna('Unknown', inplace=True)
# Replace missing comments with 'No comment'
df_maritime_scrub_rerun['comments'].fillna('No comment', inplace=True)
df_maritime_scrub_rerun.isnull().sum()
df_maritime_scrub_rerun.head(10)
df_maritime_scrub_rerun.info()


<class 'pandas.core.frame.DataFrame'>
Index: 80252 entries, 0 to 80263
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   datetime            80252 non-null  object 
 1   city                80252 non-null  object 
 2   state               80252 non-null  object 
 3   country             80252 non-null  object 
 4   shape               80252 non-null  object 
 5   duration_seconds    80252 non-null  object 
 6   duration_hours_min  80252 non-null  object 
 7   comments            80252 non-null  object 
 8   date_posted         80252 non-null  object 
 9   latitude            80252 non-null  object 
 10  longitude           80252 non-null  float64
dtypes: float64(1), object(10)
memory usage: 7.3+ MB


<ipython-input-6-884318495>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_maritime_scrub_rerun['shape'].fillna('Unknown', inplace=True)
<ipython-input-6-884318495>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', 

In [ ]:
# Set all country values to be lower case and strip any training spaces
df_maritime_scrub_rerun['country'] = df_maritime_scrub_rerun['country'].str.strip().str.lower()

# Regex to remove everything in parenthesis including the parenthesis
df_maritime_scrub_rerun['city'] = df_maritime_scrub_rerun['city'].str.replace(r'\(.*\)', '', regex=True)

# Strip to remove leading/trailing spaces that might be left after removal
df_maritime_scrub_rerun['city'] = df_maritime_scrub_rerun['city'].str.strip()

# Display the results
print(df_maritime_scrub_rerun.info())

<class 'pandas.core.frame.DataFrame'>
Index: 80252 entries, 0 to 80263
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   datetime            80252 non-null  object 
 1   city                80252 non-null  object 
 2   state               80252 non-null  object 
 3   country             80252 non-null  object 
 4   shape               80252 non-null  object 
 5   duration_seconds    80252 non-null  object 
 6   duration_hours_min  80252 non-null  object 
 7   comments            80252 non-null  object 
 8   date_posted         80252 non-null  object 
 9   latitude            80252 non-null  object 
 10  longitude           80252 non-null  float64
dtypes: float64(1), object(10)
memory usage: 7.3+ MB
None


In [ ]:
df_maritime_scrub_rerun[df_maritime_scrub_rerun['state'].isnull()]


,datetime,city,state,country,shape,duration_seconds,duration_hours_min,comments,date_posted,latitude,longitude


## **Step 2.4 Dealing with parenthesis in state column names.**

While we managed to clean the entire dataset from missing values, there is still a couple of issue i've noticed and would like to address.

1.   There are parenthesis in 'State' names which i would like to not be present. We have already completed this step for our City names so it's just a matter of repeating this step.

2.   When we reverse geo located the 'State' for our record the state is returned as a full string not abbreviation. This is going to be abit more tricky as our dataset spans entire world, but i am sure we can figure something out. ** Unfortunately there is no easy way to fix this. That i know of at the moment. **




In [ ]:
# Regex to remove everything in parenthesis including the parenthesis
df_maritime_scrub_rerun['state'] = df_maritime_scrub_rerun['state'].str.replace(r'\(.*\)', '', regex=True)

# Strip to remove leading/trailing spaces that might be left after removal
df_maritime_scrub_rerun['state'] = df_maritime_scrub_rerun['state'].str.strip()

So now we have our missing values filled whether it's state, country missing shape or comments. Let's explore the dataset a little bit further to identify any other potential issues we might have.

In [ ]:
df_maritime_scrub_rerun.head(10)

,datetime,city,state,country,shape,duration_seconds,duration_hours_min,comments,date_posted,latitude,longitude
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.941111
1,10/10/1949 21:00,lackland afb,tx,us,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.38421,-98.581082
2,10/10/1955 17:00,chester,England,gb,circle,20,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.2,-2.916667
3,10/10/1956 21:00,edna,tx,us,circle,20,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.9783333,-96.645833
4,10/10/1960 20:00,kaneohe,hi,us,light,900,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.4180556,-157.803611
5,10/10/1961 19:00,bristol,tn,us,sphere,300,5 minutes,My father is now 89 my brother 52 the girl wit...,4/27/2007,36.595,-82.188889
6,10/10/1965 21:00,penarth,Wales,gb,circle,180,about 3 mins,penarth uk circle 3mins stayed 30ft above m...,2/14/2006,51.434722,-3.180000
7,10/10/1965 23:45,norwalk,ct,us,disk,1200,20 minutes,A bright orange color changing to reddish colo...,10/02/1999,41.1175,-73.408333
8,10/10/1966 20:00,pell city,al,us,disk,180,3 minutes,Strobe Lighted disk shape object observed clos...,3/19/2009,33.5861111,-86.286111
9,10/10/1966 21:00,live oak,fl,us,disk,120,several minutes,Saucer zaps energy from powerline as my pregna...,05/11/2005,30.2947222,-82.984167


## **Step 2.5 Dealing with dates, changing dates from mixed to UK format**

First thing i would like to do is to check our datetime column for any values that are not actually date. We can do this by attempting to convert the column in to date format and see which values fail to convert. Let's create a copy of our dataset on do that.

In [ ]:
df_datetime = df_maritime_scrub_rerun.copy()
df_datetime['datetime'] = pd.to_datetime(df_datetime['datetime'], errors='coerce')
df_datetime.info()

<class 'pandas.core.frame.DataFrame'>
Index: 80252 entries, 0 to 80263
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   datetime            79561 non-null  datetime64[ns]
 1   city                80252 non-null  object        
 2   state               80252 non-null  object        
 3   country             80252 non-null  object        
 4   shape               80252 non-null  object        
 5   duration_seconds    80252 non-null  object        
 6   duration_hours_min  80252 non-null  object        
 7   comments            80252 non-null  object        
 8   date_posted         80252 non-null  object        
 9   latitude            80252 non-null  object        
 10  longitude           80252 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(9)
memory usage: 9.4+ MB


We can see after changing the column to datetime we have some records that failed to convert. We will remove these records from our dateset

In [ ]:
df_datetime = df_datetime.dropna(subset=['datetime'])
df_datetime.info()

<class 'pandas.core.frame.DataFrame'>
Index: 79561 entries, 0 to 80263
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   datetime            79561 non-null  datetime64[ns]
 1   city                79561 non-null  object        
 2   state               79561 non-null  object        
 3   country             79561 non-null  object        
 4   shape               79561 non-null  object        
 5   duration_seconds    79561 non-null  object        
 6   duration_hours_min  79561 non-null  object        
 7   comments            79561 non-null  object        
 8   date_posted         79561 non-null  object        
 9   latitude            79561 non-null  object        
 10  longitude           79561 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(9)
memory usage: 7.3+ MB


Let's take a look at our date_posted column.

In [ ]:
df_datetime['date_posted'].head(25)

,date_posted
0,4/27/2004
1,12/16/2005
2,1/21/2008
3,1/17/2004
4,1/22/2004
5,4/27/2007
6,2/14/2006
7,10/02/1999
8,3/19/2009
9,05/11/2005


We can quickly can identify that some of the dates are recorded in us format, some of the dates are recorded in uk format which is not good to us. We will convert dates in to UK format by attempting conversion in to both UK and US format, and then combining the results


In [ ]:
# Make a copy of the dataframe
df_posted_date = df_datetime.copy()

# First we will create a column which attempts to convert dates in to uk format
df_posted_date['date_posted_temp_uk_parsed'] = pd.to_datetime(
    df_posted_date['date_posted'], dayfirst=True, errors='coerce'
)

# We will also create a column that attempts to convert values in to us format
df_posted_date['date_posted_temp_us_parsed'] = pd.to_datetime(
    df_posted_date['date_posted'], dayfirst=False, errors='coerce'
)

# We will combine both columns together
df_posted_date['date_posted_cleaned_dt'] = \
    df_posted_date['date_posted_temp_uk_parsed'].fillna(
    df_posted_date['date_posted_temp_us_parsed']
)

<ipython-input-14-4108216906>:3: UserWarning: Parsing dates in %m/%d/%Y format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df_posted_date['date_posted_temp_uk_parsed'] = pd.to_datetime(


In [ ]:
# Apply required formatting
df_posted_date['date_posted_uk_format'] = \
    df_posted_date['date_posted_cleaned_dt'].dt.strftime('%d/%m/%Y')

In [ ]:
# Replace our original date_posted column with a new combined column
df_posted_date['date_posted'] = df_posted_date['date_posted_uk_format']

In [ ]:
# And finally remove our temporary columns
df_posted_date = df_posted_date.drop(columns=[
    'date_posted_temp_uk_parsed',
    'date_posted_temp_us_parsed',
    'date_posted_cleaned_dt',
    'date_posted_uk_format'
])

In [ ]:
df_posted_date['date_posted'] = pd.to_datetime(df_posted_date['date_posted'], errors='coerce')

<ipython-input-48-1995537283>:1: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_posted_date['date_posted'] = pd.to_datetime(df_posted_date['date_posted'], errors='coerce')


In [ ]:
df_posted_date.info()

<class 'pandas.core.frame.DataFrame'>
Index: 79561 entries, 0 to 80263
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   datetime            79561 non-null  datetime64[ns]
 1   city                79561 non-null  object        
 2   state               79561 non-null  object        
 3   country             79561 non-null  object        
 4   shape               79561 non-null  object        
 5   duration_seconds    79561 non-null  object        
 6   duration_hours_min  79561 non-null  object        
 7   comments            79561 non-null  object        
 8   date_posted         79561 non-null  datetime64[ns]
 9   latitude            79561 non-null  object        
 10  longitude           79561 non-null  float64       
dtypes: datetime64[ns](2), float64(1), object(8)
memory usage: 9.3+ MB


## **Step 2.6 Dealing with sighting durations**

We are almost done cleaning the dataset.I would like to take a lookg at is our duration_seconds and duration_hours_min columns. We have duration in seconds but our duration_hours_min has mixed formatting. For example sometimes it says 15 minutes, sometimes 1 - 2 hours etc.

In [ ]:
# Let's first copy our previous dataframe
df_encounter_duration = df_posted_date.copy()

# Let's convert the duration_seconds to a numeric data type to see if there is missing values,
df_encounter_duration['duration_seconds_cleaned'] = pd.to_numeric(df_encounter_duration['duration_seconds'], errors='coerce')

# Let's see if there is any missing values
df_encounter_duration.info()

# Let's investigate the values that have failed to convert
invalid_duration_entries = df_encounter_duration[df_encounter_duration['duration_seconds_cleaned'].isna()]

# If there is invalid entries tell me
if not invalid_duration_entries.empty:
    print("These are the entries that failed the initial numeric conversion:")
    print(invalid_duration_entries['duration_seconds'])
    print(f"\nTotal records that failed initial conversion: {len(invalid_duration_entries)}")
else:
    print("\nAll entries successfully converted in the first attempt.")




<class 'pandas.core.frame.DataFrame'>
Index: 79561 entries, 0 to 80263
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   datetime                  79561 non-null  datetime64[ns]
 1   city                      79561 non-null  object        
 2   state                     79561 non-null  object        
 3   country                   79561 non-null  object        
 4   shape                     79561 non-null  object        
 5   duration_seconds          79561 non-null  object        
 6   duration_hours_min        79561 non-null  object        
 7   comments                  79561 non-null  object        
 8   date_posted               79561 non-null  datetime64[ns]
 9   latitude                  79561 non-null  object        
 10  longitude                 79561 non-null  float64       
 11  duration_seconds_cleaned  79558 non-null  float64       
dtypes: datetime64[ns](2), f

In [ ]:
# In order to remove string values we first need to convert them to string type to you str methods
df_encounter_duration['duration_seconds_cleaned'] = df_encounter_duration['duration_seconds'].astype(str)

# Let's remove the backtics using str.replace and str.strip to replace any left over spaces
df_encounter_duration['duration_seconds_cleaned'] = (
    df_encounter_duration['duration_seconds_cleaned']
    .str.replace('`', '', regex=False)
    .str.strip()
)



In [ ]:
df_encounter_duration['duration_seconds_cleaned'] = pd.to_numeric(df_encounter_duration['duration_seconds_cleaned'], errors='coerce')
df_encounter_duration.info()

<class 'pandas.core.frame.DataFrame'>
Index: 79561 entries, 0 to 80263
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   datetime                  79561 non-null  datetime64[ns]
 1   city                      79561 non-null  object        
 2   state                     79561 non-null  object        
 3   country                   79561 non-null  object        
 4   shape                     79561 non-null  object        
 5   duration_seconds          79561 non-null  object        
 6   duration_hours_min        79561 non-null  object        
 7   comments                  79561 non-null  object        
 8   date_posted               79561 non-null  datetime64[ns]
 9   latitude                  79561 non-null  object        
 10  longitude                 79561 non-null  float64       
 11  duration_seconds_cleaned  79561 non-null  float64       
dtypes: datetime64[ns](2), f

In [ ]:
df_encounter_duration['duration_seconds'] = df_encounter_duration['duration_seconds_cleaned']
df_encounter_duration = df_encounter_duration.drop(columns=[
    'duration_seconds_cleaned'
])



In [ ]:
df_encounter_duration.info()

<class 'pandas.core.frame.DataFrame'>
Index: 79561 entries, 0 to 80263
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   datetime            79561 non-null  datetime64[ns]
 1   city                79561 non-null  object        
 2   state               79561 non-null  object        
 3   country             79561 non-null  object        
 4   shape               79561 non-null  object        
 5   duration_seconds    79561 non-null  float64       
 6   duration_hours_min  79561 non-null  object        
 7   comments            79561 non-null  object        
 8   date_posted         79561 non-null  datetime64[ns]
 9   latitude            79561 non-null  object        
 10  longitude           79561 non-null  float64       
dtypes: datetime64[ns](2), float64(2), object(7)
memory usage: 9.3+ MB


Since we have duration_seconds column we will replace values in duration_hours_min by converting seconds in to hours minutes float (ie. 1.5 is 1 hours 30 minutes). This is due to values stored in duration_hours_min all having different format.

In [ ]:
# Convert seconds to hours and minutes
df_encounter_duration['duration_hours_min'] = (df_encounter_duration['duration_seconds'] / 3600).round(3)

In [ ]:
pd.to_numeric(df_encounter_duration['duration_hours_min'], errors='coerce')

,duration_hours_min
0,0.750
1,2.000
2,0.006
3,0.006
4,0.250
...,...
80259,0.167
80260,0.333
80261,0.333
80262,0.001


Finally i have noticed that our latitude is stills tored as object, instead of float lets fix that

In [ ]:
df_encounter_duration.info()

<class 'pandas.core.frame.DataFrame'>
Index: 79561 entries, 0 to 80263
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   datetime            79561 non-null  datetime64[ns]
 1   city                79561 non-null  object        
 2   state               79561 non-null  object        
 3   country             79561 non-null  object        
 4   shape               79561 non-null  object        
 5   duration_seconds    79561 non-null  float64       
 6   duration_hours_min  79561 non-null  float64       
 7   comments            79561 non-null  object        
 8   date_posted         79561 non-null  datetime64[ns]
 9   latitude            79561 non-null  float64       
 10  longitude           79561 non-null  float64       
dtypes: datetime64[ns](2), float64(4), object(5)
memory usage: 9.3+ MB


This is something that should have been done previously.

In [ ]:
# Fixing data type of latitude
df_encounter_duration['latitude'] = df_encounter_duration['latitude'].astype(float)

In [ ]:
df_encounter_duration.info()

<class 'pandas.core.frame.DataFrame'>
Index: 79561 entries, 0 to 80263
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   datetime            79561 non-null  datetime64[ns]
 1   city                79561 non-null  object        
 2   state               79561 non-null  object        
 3   country             79561 non-null  object        
 4   shape               79561 non-null  object        
 5   duration_seconds    79561 non-null  float64       
 6   duration_hours_min  79561 non-null  float64       
 7   comments            79561 non-null  object        
 8   date_posted         79561 non-null  datetime64[ns]
 9   latitude            79561 non-null  float64       
 10  longitude           79561 non-null  float64       
dtypes: datetime64[ns](2), float64(4), object(5)
memory usage: 9.3+ MB


## **Step 2.7 Dealing with HTML entities and Non ASCII characters from comments**

Let's take a look at our comments.

In [ ]:
df_encounter_duration.head(10)

,datetime,city,state,country,shape,duration_seconds,duration_hours_min,comments,date_posted,latitude,longitude
0,1949-10-10 20:30:00,san marcos,tx,us,cylinder,2700.0,0.750,This event took place in early fall around 194...,2004-04-27,29.883056,-97.941111
1,1949-10-10 21:00:00,lackland afb,tx,us,light,7200.0,2.000,1949 Lackland AFB&#44 TX. Lights racing acros...,2005-12-16,29.384210,-98.581082
2,1955-10-10 17:00:00,chester,England,gb,circle,20.0,0.006,Green/Orange circular disc over Chester&#44 En...,2008-01-21,53.200000,-2.916667
3,1956-10-10 21:00:00,edna,tx,us,circle,20.0,0.006,My older brother and twin sister were leaving ...,2004-01-17,28.978333,-96.645833
4,1960-10-10 20:00:00,kaneohe,hi,us,light,900.0,0.250,AS a Marine 1st Lt. flying an FJ4B fighter/att...,2004-01-22,21.418056,-157.803611
5,1961-10-10 19:00:00,bristol,tn,us,sphere,300.0,0.083,My father is now 89 my brother 52 the girl wit...,2007-04-27,36.595000,-82.188889
6,1965-10-10 21:00:00,penarth,Wales,gb,circle,180.0,0.050,penarth uk circle 3mins stayed 30ft above m...,2006-02-14,51.434722,-3.180000
7,1965-10-10 23:45:00,norwalk,ct,us,disk,1200.0,0.333,A bright orange color changing to reddish colo...,1999-10-02,41.117500,-73.408333
8,1966-10-10 20:00:00,pell city,al,us,disk,180.0,0.050,Strobe Lighted disk shape object observed clos...,2009-03-19,33.586111,-86.286111
9,1966-10-10 21:00:00,live oak,fl,us,disk,120.0,0.033,Saucer zaps energy from powerline as my pregna...,2005-05-11,30.294722,-82.984167


As we can see from first 10 records in the dataframe we have html entities present in the comments, which i would like to remove.

In [ ]:
import html

df_html_entities = df_encounter_duration.copy()
# Apply HTML unescaping to the comments column

df_html_entities['comments'] = df_html_entities['comments'].apply(lambda x: html.unescape(x) if isinstance(x, str) else x)

df_html_entities.head(10)

,datetime,city,state,country,shape,duration_seconds,duration_hours_min,comments,date_posted,latitude,longitude
0,1949-10-10 20:30:00,san marcos,tx,us,cylinder,2700.0,0.750,This event took place in early fall around 194...,2004-04-27,29.883056,-97.941111
1,1949-10-10 21:00:00,lackland afb,tx,us,light,7200.0,2.000,"1949 Lackland AFB, TX. Lights racing across t...",2005-12-16,29.384210,-98.581082
2,1955-10-10 17:00:00,chester,England,gb,circle,20.0,0.006,"Green/Orange circular disc over Chester, England",2008-01-21,53.200000,-2.916667
3,1956-10-10 21:00:00,edna,tx,us,circle,20.0,0.006,My older brother and twin sister were leaving ...,2004-01-17,28.978333,-96.645833
4,1960-10-10 20:00:00,kaneohe,hi,us,light,900.0,0.250,AS a Marine 1st Lt. flying an FJ4B fighter/att...,2004-01-22,21.418056,-157.803611
5,1961-10-10 19:00:00,bristol,tn,us,sphere,300.0,0.083,My father is now 89 my brother 52 the girl wit...,2007-04-27,36.595000,-82.188889
6,1965-10-10 21:00:00,penarth,Wales,gb,circle,180.0,0.050,penarth uk circle 3mins stayed 30ft above m...,2006-02-14,51.434722,-3.180000
7,1965-10-10 23:45:00,norwalk,ct,us,disk,1200.0,0.333,A bright orange color changing to reddish colo...,1999-10-02,41.117500,-73.408333
8,1966-10-10 20:00:00,pell city,al,us,disk,180.0,0.050,Strobe Lighted disk shape object observed clos...,2009-03-19,33.586111,-86.286111
9,1966-10-10 21:00:00,live oak,fl,us,disk,120.0,0.033,Saucer zaps energy from powerline as my pregna...,2005-05-11,30.294722,-82.984167


Since we had html entities present in the comments, lets as a precation try to remove any non english/standard characters from our comments.

In [ ]:
# Copy our dataframe from previous step
df_non_standard_characters = df_html_entities.copy()


We will do this in two separate steps. First we will apply Unicode Normalisation, and then remove non-ascii characters

In [ ]:
import unicodedata

df_non_standard_characters['comments'] = df_non_standard_characters['comments'].apply(lambda x: unicodedata.normalize('NFKD', x) if isinstance(x, str) else x)

In [ ]:
df_non_standard_characters['comments'] = df_non_standard_characters['comments'].apply(lambda x: ''.join(char for char in x if ord(char) < 128) if isinstance(x, str) else x)

# **Step 3. Saving cleaned dataset and comparing to original**

Finally lets save our cleaned dataset


In [ ]:
# Lets save the dataset to to avoid having to rerun the operation
# Save the resulting dataset
output_file_path = './drive/MyDrive/datasets/df_clean.csv'
df_non_standard_characters.to_csv(output_file_path, index=False) # index=False is important

In [ ]:
df_clean = pd.read_csv('./drive/MyDrive/datasets/df_clean.csv')

In [ ]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79561 entries, 0 to 79560
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   datetime            79561 non-null  object 
 1   city                79561 non-null  object 
 2   state               79561 non-null  object 
 3   country             79561 non-null  object 
 4   shape               79561 non-null  object 
 5   duration_seconds    79561 non-null  float64
 6   duration_hours_min  79561 non-null  float64
 7   comments            79561 non-null  object 
 8   date_posted         79561 non-null  object 
 9   latitude            79561 non-null  float64
 10  longitude           79561 non-null  float64
dtypes: float64(4), object(7)
memory usage: 6.7+ MB


Finally let's compare our original df with a resulting df_clean

In [ ]:
percentage_lost = ((len(df) - len(df_clean)) / len(df)) * 100
print("Original row count:", len(df))
print("Cleaned row count:", len(df_clean))
print("Records lost:", len(df) - len(df_clean))
print(f"Percentage of records lost: {percentage_lost:.2f}%")

Original row count: 80332
Cleaned row count: 79561
Records lost: 771
Percentage of records lost: 0.96%
